DESARROLLO API

El objetivo es disponibilizar los datos de la empresa usando el framework FastAPI. Las consultas son las siguientes: 6 funciones para los endpoints que se consumirán en la API

def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en el mes de X

def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X

def score_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.
                    Ejemplo de retorno: La película X fue estrenada en el año X con un score/popularidad de X

def votos_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.
                    Ejemplo de retorno: La película X fue estrenada en el año X. La misma cuenta con un total de X valoraciones, con un promedio de X

def get_actor( nombre_actor ): Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.
                    Ejemplo de retorno: El actor X ha participado de X cantidad de filmaciones, el mismo ha conseguido un retorno de X con un promedio de X por filmación

def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils.extmath import randomized_svd
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
movies_completo = pd.read_csv('Movies_completo.csv')

In [3]:
tipo_dato = movies_completo['ReleaseDate'].dtypes
print(tipo_dato)

object


In [4]:
#Convierto la columna ReleaseDate en tipo de dato fecha y hora
#Coerce es para errores que puedan ocurrir durante la conversión, los datos se establecen como NaT

movies_completo['ReleaseDate'] = pd.to_datetime(movies_completo['ReleaseDate'], errors='coerce')

In [5]:
movies_completo['ReleaseMonth'] = pd.to_datetime(movies_completo['ReleaseMonth'], errors='coerce')

In [6]:
def cantidad_filmaciones_mes(mes: str):
    data = movies_completo

    meses = {
        'enero': 1,
        'febrero': 2,
        'marzo': 3,
        'abril': 4,
        'mayo': 5,
        'junio': 6,
        'julio': 7,
        'agosto': 8,
        'septiembre': 9,
        'octubre': 10,
        'noviembre': 11,
        'diciembre': 12
    }

    num_mes = meses.get(mes.lower())

    if num_mes is None:
        raise ValueError('Dato mes inexistente')

    # Convertir la columna 'ReleaseDate' a tipo datetime
    data['ReleaseMonth'] = pd.to_datetime(data['ReleaseMonth'], errors='coerce')

    filmaciones_mes = data[data['ReleaseMonth'].dt.month == num_mes]

    cantidad_filmaciones = len(filmaciones_mes)

    return f"Se estrenaron {cantidad_filmaciones} filmaciones el mes {mes.capitalize()}."

In [7]:
#Pruebo la función 

cantidad_filmaciones_mes('enero')

'Se estrenaron 45376 filmaciones el mes Enero.'

In [8]:
def cantidad_filmaciones_dia(Dia):
    data = movies_completo
    #Establezco un diccionario con los nombres de los días en español
    dias = {
        'lunes': 0,
        'martes': 1,
        'miercoles': 2,
        'jueves': 3,
        'viernes': 4,
        'sabado': 5,
        'domingo': 6
    }

    num_dia = dias.get(Dia.lower()) #Obtengo el número de día y busca el mismo en el diccionario de arriba

    if num_dia is None:
        raise ValueError('Dato día inexistente')

    filmaciones_dia = data[data['ReleaseDate'].dt.weekday == num_dia]   #Filtro las filmaciones por día de la semana

    cantidad_filmaciones = len(filmaciones_dia) #Calculo la cantidad de filmaciones en el día

    return f"Se estrenaron {cantidad_filmaciones} filmaciones los días {Dia.capitalize()}." #Devuelvo el día con la primer letra en mayúscula


In [9]:
#Pruebo la función

cantidad_filmaciones_dia ('lunes')

'Se estrenaron 3503 filmaciones los días Lunes.'

In [10]:
def score_titulo(titulo_de_la_filmación):
    data = movies_completo #creo la variable data que contiene el dataframe movies_completo

    titulo = titulo_de_la_filmación.title()  #Capitalizo la primera letra de cada palabra en el título proporcionado y lo asigno a la variable título
    match = data[data['Title'] == titulo]    #Recorro data para encontrar dónde en la columna 'title' esté exactamente el valor de título. 

    #Si match no está vacía, obtengo los valores del título, año de lanzamiento y puntuación de popularidad. Y asigno cada resultado a una variable para llamarlas en el return
    if not match.empty:
        titulo1 = match['Title'].values[0]
        anio = match['ReleaseYear'].values[0]
        score_popularity = match['Popularity'].values[0]

        score_popularity = round(score_popularity, 2) #Redondeo la popularidad a dos decimales. 
            
        return f"La película {titulo1} fue estrenada en el año {anio} con un score/popularidad de {score_popularity}."

    #Si no hay match
    return "No se encontró la película especificada."

In [11]:
#Pruebo la función

score_titulo('A Brief Vacation')

'La película A Brief Vacation fue estrenada en el año 1973.0 con un score/popularidad de 0.42.'

In [12]:
def votos_titulo(titulo_de_la_filmación):
    data = movies_completo  #Inicio la variable data a partir del data frame 
    data['Title'] = data['Title'].str.title() # Realizo el cambio de todos los valores de la columna title a mayuscula inicial
    titulo = titulo_de_la_filmación.title() #Lo mismo con título

    if (data['Title'] == titulo).any(): #Verifico si el título coincide con algún título de la columna 'Title' del dataframe
        peliculas = data[data['Title'] == titulo] #Obtengo todas las películas que coinciden con el título
        primera_pelicula = peliculas.iloc[0] #Tomo sólo la primera

        qvotos = primera_pelicula['VoteCount']
        
        if qvotos < 2000:
            return "La película no cuenta con al menos 2000 valoraciones."
        
        else:
            anio = primera_pelicula['ReleaseYear'] #Almaceno el año de lanzamiento 
            #qvotos = data[data['Title'] == titulo]['VoteCount'].item() #Lo mismo con la cantidad de votos
            promedio = primera_pelicula['VoteAverage'] #Repito con el promedio de votos

        return f"La película {titulo} fue estrenada en el año {anio}. La misma cuenta con un total de {qvotos} valoraciones, con un promedio de {promedio}."
    
    else:
        return "La película no se encontró."

In [13]:
#Pruebo la función

votos_titulo('Batman')

'La película Batman fue estrenada en el año 1989.0. La misma cuenta con un total de 2145.0 valoraciones, con un promedio de 7.0.'

In [14]:
def get_actor(nombre_actor):
    actor = movies_completo[movies_completo['ActorName'].apply(lambda x: isinstance(x, str) and nombre_actor in x)] #creo un DataFrame 'actor' con todas las filas de movies_completo, la columna 'ActorName' contiene el nombre del actor especificado. Con apply en la columna 'ActorName', aplico lambda para verificar si x es una cadena y si nombre_actor se encuentra dentro de esa cadena
    if len (actor) == 0:
        print ("Este integrante no se encuentra en la lista")
        return
    
    qpelis = len(actor)  #Longitud de la lista que se arma con lambda
    retorno = actor['Return'].sum()
    promretorno = retorno / qpelis
    
    return f"El/La actor/actriz {nombre_actor} ha participado de {qpelis} filmaciones, ha conseguido un retorno de {retorno} con un promedio de {promretorno} por filmación"

In [15]:
#Pruebo la función

get_actor('Vanda Duarte')

'El/La actor/actriz Vanda Duarte ha participado de 3 filmaciones, ha conseguido un retorno de 8.377430547021945 con un promedio de 2.792476849007315 por filmación'

In [20]:
def get_director(director: str):
    ''' Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. 
    Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.'''
    
    #Creo la lista en la que se guardan los detalles de la película dirigida por el 'director'
    pelis_director = []
    for index, row in movies_completo.iterrows():
        if director in row['Director']:
            #Creo una cadena de texto que contiene los detalles de la película
            pelicula = '- ' + row['Title'] + ':\n'
            #Si la fecha de lanzamiento no es nula se agrega una línea adicional a la cadena 'película'
            if isinstance(row['ReleaseDate'], str) and pd.notnull(row['ReleaseDate']):
                release_date = pd.to_datetime(row['ReleaseDate'])
                pelicula += '  - Lanzamiento: ' + release_date.strftime('%Y-%m-%d') + '\n'
            pelicula += '  - Retorno: ' + "{:.2f}".format(row['Return']) + '\n' + \
                        '  - Costo: ' + "{:.2f}".format(row['Budget']) + '\n' + \
                        '  - Ganancia: ' + "{:.2f}".format(row['Revenue']) + '\n'
            pelis_director.append(pelicula.replace('\n',''))
    
    if len(pelis_director) == 0:
        return 'No se encontraron películas para el director especificado.'

    exito = sum(row['Return'] for idx, row in movies_completo.iterrows() if director in row['Director'])
    exito = round(exito, 2)

    resultado = ['El director ' + director + ' tiene un éxito medido por retorno de ' + str(exito)]
    resultado.extend(pelis_director)

    return resultado

In [21]:
resultado_director = get_director('Quentin Tarantino')
for item in resultado_director:
    print(item)

El director Quentin Tarantino tiene un éxito medido por retorno de 32.73
- Pulp Fiction:  - Retorno: 26.74  - Costo: 8000000.00  - Ganancia: 213928762.00
- Weekend At Bernie'S:  - Retorno: 4.65  - Costo: 6500000.00  - Ganancia: 30218387.00
- Mr. Magoo:  - Retorno: 0.00  - Costo: 30000000.00  - Ganancia: 0.00
- Runaway Jury:  - Retorno: 1.34  - Costo: 60000000.00  - Ganancia: 80154140.00
- This So-Called Disaster: Sam Shepard Directs "The Late Henry Moss":  - Retorno: 0.00  - Costo: 0.00  - Ganancia: 0.00
- The Moguls:  - Retorno: 0.00  - Costo: 0.00  - Ganancia: 0.00
- Strike:  - Retorno: 0.00  - Costo: 0.00  - Ganancia: 0.00
- Minecraft: The Story Of Mojang:  - Retorno: 0.00  - Costo: 0.00  - Ganancia: 0.00
- The Wonders:  - Retorno: 0.00  - Costo: 0.00  - Ganancia: 0.00
- Steal My Heart:  - Retorno: 0.00  - Costo: 0.00  - Ganancia: 0.00


In [22]:
def recomendacion(titulo):
    data = movies_completo

    titulo = titulo.title()  #Capitalizo la primera letra de cada palabra en el título proporcionado

    #Obtengo las palabras clave del título para buscar películas similares
    palabras_clave = set(titulo.split())

    #Filtro las películas que contienen al menos una palabra clave en el título
    peliculas_similares = data[data['Title'].apply(lambda x: any(word in x for word in palabras_clave))]

    #Excluyo la película original del resultado
    peliculas_similares = peliculas_similares[peliculas_similares['Title'] != titulo]

    #Ordeno las películas por popularidad de mayor a menor
    peliculas_similares = peliculas_similares.sort_values(by='Popularity', ascending=False)

    #Selecciono las 5 primeras películas recomendadas
    peliculas_recomendadas = peliculas_similares.head(5)

    #Obtengo los títulos de las películas recomendadas
    lista_recomendada = peliculas_recomendadas['Title'].tolist()

    return {'Lista recomendada': lista_recomendada}


In [23]:
#Pruebo la función

recomendacion('The Avengers')

{'Lista recomendada': ['Beauty And The Beast',
  'Guardians Of The Galaxy Vol. 2',
  'The Hunger Games: Mockingjay - Part 1',
  'War For The Planet Of The Apes',
  'Pirates Of The Caribbean: Dead Men Tell No Tales']}